In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('', '...')))  
# Import the required modules
import numpy as np
from matplotlib import pyplot as plt
import scipy
import seaborn as sns
import pickle
import h5py
import json
import argparse
from functions.system.limbNetwork import limbNetwork
from functions.controller import controllerFuncs

## Here we will define the simulation parameters 

This involves 
1. loading the saved random network (adjacency matrix and readout weights)
2. setting the task parameters such as time for preparation and movement, and the penalties on different system states
3. setting up cost parameters

In [2]:
########## load Radnomly initialized networks from the hdf5 files ##########
def load_networks_by_spectral_radius(args):
    filename = args.filename
    spectral_radius = args.spectral_radiuses[0] + 0
    networks = []
    readouts = []
    with h5py.File(filename, 'r') as f:
        spectral_radius_group = str(spectral_radius)  # Ensure the spectral radius is in the correct format
        if spectral_radius_group in f:
            for i in range(10):  # Assuming there are always 10 networks
                network_dataset_name = f'{spectral_radius_group}/networks/network_{i}'
                readout_dataset_name = f'{spectral_radius_group}/readouts/readout_{i}'
                if network_dataset_name in f:
                    network = f[network_dataset_name][()]
                    networks.append(network)

                    readout = f[readout_dataset_name][()]
                    readouts.append(readout)
                else:
                    print(f"Dataset {network_dataset_name} not found.")
        else:
            print(f"Spectral radius group {spectral_radius_group} not found.")
    return networks, readouts


################# define simulation parameters #####################
def setsimparams(widetarget_param, shooting_param):
    k = 0.1
    m = 1
    tau = 0.06
    delta = 0.01
    rtime = 0.3
    prep_time = 1.0

    num_neurons = 100
    nStep = int(np.round((rtime + prep_time)/delta))
    nPrep = int(np.round(prep_time/delta))
    nHold = 0
    w_execute = np.zeros(shape=(8+num_neurons+1, nStep-nPrep+1))

    

    #### COSTS parameters by including the neuron_tau_net in B matrix ########
    prep_alpha = 1e3
    effort_alpha = 1e-12


    #### TESTING MORE COSTS ########
    prep_alpha = 100
    effort_alpha = 1e-11
    
    for t in range(0, nStep - nPrep):
        w_execute[:8, t] = (1/(nStep-nHold)) * np.array([100 * widetarget_param, 
                                                 0.15 * shooting_param, 0, 
                                                 0, 100, 0.15 * shooting_param, 0, 0]) * ((t) / (nStep - nPrep))**6
        w_execute[:8, t] = (1/(nStep-nHold)) * np.array([200, 0.15 * shooting_param, 0.001 , 0, 200, 0.15 * shooting_param, 0.001, 0]) * ((t) / (nStep - nPrep))**6


    wprep = np.zeros(shape=(8+num_neurons+1, nPrep+1))
    for t in range(0, nPrep+1):
        wprep[:8, t] = (1/nStep) * prep_alpha * np.array([1, 1, 1, 0, 1, 1, 1, 0]) 

    r = effort_alpha + 0.0


    simparams = {'k': k, 'm': m, 'tau': tau, 'widetarget_param': widetarget_param, 'shooting_param': shooting_param,
              'delta': delta, 'rtime': rtime, 'prep_time': prep_time, 'nStep': nStep, 'r': r, 
              'w_ffdyn': w_execute, 
              'num_neurons': num_neurons, 'nPrep': nPrep, 'wprep': wprep, 'neuron_tau_net': 20e-3}
    
    return simparams

#### define the function to simulate the reaching control task to given number of targets ###########


In [3]:
def reach_ineritalloading(params):
    # GET body matrices and SET cost matrices
    system_dynamics= limbNetwork(params)
    Atot, Btot, Htot = system_dynamics.getSystemMatrices()
    #R, Q = setCostmatrices(Atot, Btot, params['delta'], params['r'], params['w']) # cross-checked the R, Q with Fred's code. They are the same
    #R, Q = setCostmatrices_wprep(Atot, Btot, params['delta'], params['r'], params['w_ffdyn'], params['wprep'], params['nPrep']) # cross-checked the R, Q with Fred's code. They are the same

    # By this point, we have the system matrices (Atot,Btot) and the cost matrices (R,Q). Now we need to compute the feedback gains L.
    # For this, we need to compute the noise covariance matrix SigmaXi. We will use the same noise covariance matrix as Fred's code.
    # The noise covariance matrix is defined as follows:
    SigmaXi = system_dynamics.SigmaXi + 0.0
    SigmaOmega = system_dynamics.SigmaOmega + 0.0

    R, Q = controllerFuncs.setCostmatrices_wprep_MPC(Atot, Btot, params['delta'], params['r'], params['w_ffdyn'], params['wprep'], params['nPrep'])
    L = controllerFuncs.basicLQG(Atot, Btot, Q, R, SigmaXi)
    K = controllerFuncs.basicKalman(Atot, Btot, Q, R, Htot, SigmaXi, SigmaOmega)
    
    system_dynamics.SigmaXi[3,3] = 0                 # X force trick
    SigmaXi[3, 3] = 0                 # X force trick


    # Now we have the feedback gains L. We can use this to simulate the system.
    # For this, we need to define the simulation time and the target parameters
    simtime = np.arange(0, params['rtime']+params['prep_time'] + params['delta'], params['delta'])
    num_targconditions = 4
    num_states = (params['num_neurons'] + 1 + 8) * 3
    num_controls = params['num_neurons']
    num_steps = len(simtime)
    all_states = np.zeros((num_steps, num_targconditions, num_states))
    all_estimates = np.zeros((num_steps, num_targconditions, num_states))
    all_controls = np.zeros((num_steps, num_targconditions, num_controls))
    all_innovation = np.zeros((num_steps, num_targconditions, num_states))
    
    # targets location is in the indices 8 and 12 for X and Y respectively
    # so for 8 conditions, we will use just one vertical target but first 4 conds are unpertuebd movements and the last 4 are perturbed
    # the target state is defined as follows:
    target_states = np.zeros((num_targconditions, num_states))
    target_states[:, 8 + params['num_neurons'] + 1] = 0.0 + 0.0*np.cos(np.arange(0, 2*np.pi, 2*np.pi/num_targconditions))   # X
    target_states[:, 12 + params['num_neurons'] + 1] = 0.2 + 0.0*np.sin(np.arange(0, 2*np.pi, 2*np.pi/num_targconditions))  # Y

    # target states for preparation
    prep_states = np.zeros((num_targconditions, num_states))
    prep_states[:, 2 * (8 + params['num_neurons'] + 1)] = 0.0   # X
    prep_states[:, 2 * (8 + params['num_neurons'] + 1) + 4] = 0.0   # X

    # the target states are active for the entire simulation time
    # so the all_states matrix will have the target states repeated for all time steps
    all_states[:, :, 8 + params['num_neurons'] + 1] = target_states[:, 8 +params['num_neurons'] + 1].reshape((1, num_targconditions)).repeat(num_steps, axis=0)
    all_states[:, :, 12 + params['num_neurons'] + 1] = target_states[:, 12 + params['num_neurons'] + 1].reshape((1, num_targconditions)).repeat(num_steps, axis=0)
    all_states[:, :, 2 * (8 + params['num_neurons'] + 1)] = prep_states[:, 2 * (8 + params['num_neurons'] + 1)].reshape((1, num_targconditions)).repeat(num_steps, axis=0)
    all_states[:, :, 2 * (8 + params['num_neurons'] + 1) + 4] = prep_states[:, 2 * (8 + params['num_neurons'] + 1) + 4].reshape((1, num_targconditions)).repeat(num_steps, axis=0)
    all_estimates[:, :, 8 + params['num_neurons'] + 1] = target_states[:, 8 + params['num_neurons'] + 1].reshape((1, num_targconditions)).repeat(num_steps, axis=0)
    all_estimates[:, :, 12 + params['num_neurons'] + 1] = target_states[:, 12 + params['num_neurons'] + 1].reshape((1, num_targconditions)).repeat(num_steps, axis=0)
    all_estimates[:, :, 2 * (8 + params['num_neurons'] + 1)] = prep_states[:, 2 * (8 + params['num_neurons'] + 1)].reshape((1, num_targconditions)).repeat(num_steps, axis=0)
    all_estimates[:, :, 2 * (8 + params['num_neurons'] + 1) + 4] = prep_states[:, 2 * (8 + params['num_neurons'] + 1) + 4].reshape((1, num_targconditions)).repeat(num_steps, axis=0)

    # set the desired offset states as 1 for the preparatory and actual target states
    all_states[:, :, 8 + params['num_neurons']] = 1.0
    all_states[:, :, 2*(8 + params['num_neurons'])] = 0.0
    all_states[:, :, 3*(8 + params['num_neurons'])] = 0.0
    all_estimates[:, :, 8 + params['num_neurons']] = 1.0
    all_estimates[:, :, 2*(8 + params['num_neurons'])] = 0.0
    all_estimates[:, :, 3*(8 + params['num_neurons'])] = 0.0

    # divide L into L_local and L_body
    body_state_indices_range = [[0, 8], [109, 117], [218, 226]]
    neural_state_indices_range = [[8, 108], [117, 217], [226, 326]] # excluding the offset variable
    # generate a list of indices for the body and neural states
    body_state_indices = np.concatenate([np.arange(body_state_indices_range[i][0], body_state_indices_range[i][1]) for i in range(3)])
    neural_state_indices = np.concatenate([np.arange(neural_state_indices_range[i][0], neural_state_indices_range[i][1]) for i in range(3)])

    # set the all_neuron_components and all_body_components matrices
    all_neural_components = np.zeros((num_steps, num_targconditions, len(neural_state_indices)))
    all_body_components = np.zeros((num_steps, num_targconditions, len(neural_state_indices)))
    all_default_components = np.zeros((num_steps, num_targconditions, len(neural_state_indices))) # for default dynamics
    all_net_sys_matrix = np.zeros((num_steps, num_targconditions, len(neural_state_indices), len(neural_state_indices))) # for default dynamics



    # run forward simulation
    for i in range(num_targconditions):
        system_dynamics.reset(all_states[0, i, :])
        for t in range(num_steps-1):
            # apply mechanical perturbation lateral (OPTIONAL)
            if t >= params['nPrep'] + 10 and  t  <= params['nPrep'] + 20   and i == 2:
                system_dynamics.states[3] = 0 # setting a non-zero value applies external perturbation for 100 ms 

            current_state = all_states[t, i, :] + 0.0
            #current_estimate = all_estimates[t, i, :] + 0.0

            #L = mixedLQG(Atot, Btot, Q[t:t+30, :, :], R[t:t+30, :, :], SigmaXi, params['nPrep'])

            # apply control based on current estimate
            if t < params['nPrep']:
                # prepare until the GO cue is given
                current_control = -L[0, :, :] @ current_state
            if t >= params['nPrep']:
                # unroll the control gains for the movement period
                current_control = -L[t - params['nPrep'], :, :] @ current_state
            

            all_controls[t, i, :] = current_control + 0.0

            #update the estimate
            #next_estimate, next_innovation = bodyins.nextEstimate(K[t, :, :], current_control)
            # update the state
            nextState = system_dynamics.nextState(current_control)

            # store the states
            all_states[t+1, i, :] = nextState + 0.0
            
            # also decompose the system states into neural and body components and store them
            current_neuron_states = current_state[neural_state_indices] + 0
            current_body_states = current_state[body_state_indices] + 0
            neuron_gains = L[:, :, neural_state_indices] + 0
            body_gains = L[:, :, body_state_indices] + 0

            

            # OPTIONAL CODE: to decompose network dynamics into ongoing and behavioral terms 
            if t < params['nPrep']:
                body_term = -Btot[neural_state_indices, :] @ (body_gains[0,:,:] @ current_body_states) # must be of size (augemnted states x 1)
                # compute the (A - B L_local) x_t, forming the first component of the next state
                neural_term = (Atot[neural_state_indices, neural_state_indices] - (Btot[neural_state_indices, :] @ neuron_gains[0,:,:])) @ current_neuron_states # must be of size (augmented states x 1)

                #neural_term = body_term - ((Btot[neural_state_indices, :] @ neuron_gains[0,:,:]) @ current_neuron_states)
                default_term = Atot[neural_state_indices, neural_state_indices] @ current_neuron_states

                net_sys_matrix = Atot[neural_state_indices, neural_state_indices] - (Btot[neural_state_indices, :] @ neuron_gains[0,:,:])
            else:
                body_term = -Btot[neural_state_indices, :] @ (body_gains[t - params['nPrep'],:,:] @ current_body_states)
                neural_term = (Atot[neural_state_indices, neural_state_indices] - (Btot[neural_state_indices, :] @ neuron_gains[t - params['nPrep'],:,:])) @ current_neuron_states
                
                #neural_term = body_term - ((Btot[neural_state_indices, :] @ neuron_gains[t - params['nPrep'],:,:]) @ current_neuron_states)
                default_term = Atot[neural_state_indices, neural_state_indices] @ current_neuron_states

                net_sys_matrix = Atot[neural_state_indices, neural_state_indices] - (Btot[neural_state_indices, :] @ neuron_gains[t - params['nPrep'],:,:])
            # extract the body components from the body_term
            all_body_components[t+1, i, :] = body_term + 0
            # extract the neuron components from the neural_term
            all_neural_components[t+1, i, :] = neural_term + 0
            all_default_components[t+1, i, :] = default_term + 0
            all_net_sys_matrix[t+1, i, :, :] = net_sys_matrix + 0

            
            


        
    results = {}
    results['states'] = all_states
    results['estimates'] = all_estimates
    results['controls'] = all_controls
    results['innovations'] = all_innovation
    results['L'] = L
    results['K'] = K
    results['Atot'] = Atot
    results['Btot'] = Btot
    results['bodyins'] = system_dynamics
    results['Htot'] = Htot
    results['all_neural_components'] = all_neural_components
    results['all_body_components'] = all_body_components
    results['all_default_components'] = all_default_components
    results['all_net_sys_matrix'] = all_net_sys_matrix

    return results

#### Function to run simulation of the reaching task once #####

In [4]:
########## function to run the simulation based on user input (single or batch runs) ##########
def run_simulation(args):
    spectral_radius = args.spectral_radiuses[0]
    networks, readouts = load_networks_by_spectral_radius(args)
    network_id = args.network_id + 0


    
    widetarget_levels = [1, 1, 1, 1, 1] # 
    shooting_levels = [0, 25, 50, 75, 100] # 
    shooting_levels = [0, 2.5, 5, 7.5, 10]
    simparams_context1 = setsimparams(widetarget_levels[4], shooting_levels[4])
    simparams_context2 = setsimparams(widetarget_levels[3], shooting_levels[3])
    simparams_context3 = setsimparams(widetarget_levels[2], shooting_levels[2])
    simparams_context4 = setsimparams(widetarget_levels[1], shooting_levels[1])
    simparams_context5 = setsimparams(widetarget_levels[0], shooting_levels[0])
    
    network = networks[network_id]
    readout = readouts[network_id]
    networkparams = {}
    networkparams['Wrec'] = network
    networkparams['Wout'] = readout
    networkparams['spectral_radius'] = spectral_radius
    networkparams['network_id'] = network_id

    networkparams['Win'] = np.zeros((networkparams['Wrec'].shape[0], 8))
    S = 8
    N = networkparams['Wrec'].shape[0]

    
    simparams_context1.update(networkparams)
    simparams_context2.update(networkparams)
    simparams_context3.update(networkparams)
    simparams_context4.update(networkparams)
    simparams_context5.update(networkparams)

    # run the simulations
    results_shootarget_lvl1 = reach_ineritalloading(simparams_context1)
    results_shootarget_lvl2 = reach_ineritalloading(simparams_context2)
    results_shootarget_lvl3 = reach_ineritalloading(simparams_context3)
    results_shootarget_lvl4 = reach_ineritalloading(simparams_context4)
    results_shootarget_lvl5 = reach_ineritalloading(simparams_context5)
    return results_shootarget_lvl1, results_shootarget_lvl2, results_shootarget_lvl3, results_shootarget_lvl4, results_shootarget_lvl5

In [5]:
########## function to save the results in a new hdf5 file ##########
def save_results_to_hdf5(filename, results_shootarget_lvl1, results_shootarget_lvl2, results_shootarget_lvl3, results_shootarget_lvl4, results_shootarget_lvl5, args):
    spectral_radius = args.spectral_radiuses[0]
    i = args.network_id + 0
    with h5py.File(filename, 'a') as f:  # Change 'w' to 'a' to append to the file instead of overwriting
            group = f.require_group(str(spectral_radius))  # Use require_group instead of create_group
            if f'network_{i}' in group: # check if the group for network i already exists
                network_group = group[f'network_{i}']
            else:
                network_group = group.create_group(f'network_{i}')

            # Function to save a dictionary of ndarrays
            def save_dict_of_ndarrays(subgroup, data_dict):
                for key, value in data_dict.items():
                    # Ensure value is an ndarray here; if not, you might need additional handling
                    if isinstance(value, np.ndarray):
                        subgroup.create_dataset(key, data=value)
                    else:
                        # Handle non-ndarray data types (e.g., serialize or convert)
                        print(f"Skipping {key}: not an ndarray")

            # Save each results variable
            save_dict_of_ndarrays(network_group.create_group('results_shootarget_lvl1'), results_shootarget_lvl1)
            save_dict_of_ndarrays(network_group.create_group('results_shootarget_lvl2'), results_shootarget_lvl2)
            save_dict_of_ndarrays(network_group.create_group('results_shootarget_lvl3'), results_shootarget_lvl3)
            save_dict_of_ndarrays(network_group.create_group('results_shootarget_lvl4'), results_shootarget_lvl4)
            save_dict_of_ndarrays(network_group.create_group('results_shootarget_lvl5'), results_shootarget_lvl5)
            

In [ ]:
########## MAIN function that runs the task simulations (this example script runs one random newtork) ##########


##___________________To use the inhibitory stabilized network structure use the following args___________##
parser = argparse.ArgumentParser(description='Generate ISN neural networks with different spectral radiuses and save them to a HDF5 file.')
parser.add_argument('--filename', type=str, default = 'isn_networks.hdf5', help='The HDF5 file to save the networks to.')

#_________define the spectral radius or variance parameter for random connectivity___________#
# CAUTION: Note that for denseRNNs, although we call this parameter as spectral_radiuses, it is actually the variance scaling factor 'g'
# If using denseRNNs, set this value to 0.8 to load from existing datastore files
parser.add_argument('--spectral_radiuses', type=float, nargs='+', default=[0.3], help='The spectral radiuses of the networks.')

parser.add_argument('--seed', type=int, default=0, help='The seed for the random number generator.')
parser.add_argument('--network_id', type=int, default=0, help='The ID of the network to run the simulation for.')
parser.add_argument('--neural_effort_scaling', type=int, default=1e-12, help='The neural effort level. 1 for low, 2 for high.')
# In a Jupyter notebook avoid parsing kernel args: use empty list to get defaults
args = parser.parse_args([])
# the file should be in the data store directory above the current directory
# use only the basename from the provided filename to avoid accidental absolute paths
args.filename = os.path.join('datastore', 'WeightsData', 'ISN', os.path.basename(args.filename))
args.neural_effort_scaling = 1
store_data_filename = 'datastore/SimulationData/shootarget_task/denseRNN/denseRNN_results_singlesimulation.hdf5'


##___________________To use the dense RNN without any inhibitory stabilized network structure use the following args___________##
#parser = argparse.ArgumentParser(description='Generate dense neural networks with different spectral radiuses and save them to a HDF5 file.')
#parser.add_argument('--filename', type=str, default = 'dense_network_weights.hdf5', help='The HDF5 file to save the networks to.')

##_________define the spectral radius or variance parameter for random connectivity___________#
## CAUTION: Note that for denseRNNs, although we call this parameter as spectral_radiuses, it is actually the variance scaling factor 'g'
## If using denseRNNs, set this value to 0.8 to load from existing datastore files
#parser.add_argument('--spectral_radiuses', type=float, nargs='+', default=[0.8], help='The spectral radiuses of the networks.')

#parser.add_argument('--seed', type=int, default=0, help='The seed for the random number generator.')
#parser.add_argument('--network_id', type=int, default=0, help='The ID of the network to run the simulation for.')
#parser.add_argument('--neural_effort_scaling', type=int, default=1e-12, help='The neural effort level. 1 for low, 2 for high.')
## In a Jupyter notebook avoid parsing kernel args: use empty list to get defaults
#args = parser.parse_args([])
## the file should be in the data store directory above the current directory
## use only the basename from the provided filename to avoid accidental absolute paths
#args.filename = os.path.join('datastore', 'WeightsData', 'denseRNN', os.path.basename(args.filename))
#args.neural_effort_scaling = 1
#store_data_filename = 'datastore/SimulationData/shootarget_task/denseRNN/denseRNN_results_singlesimulation.hdf5'
	



# Run a single simulation
args.network_id = 0
results_shootarget_lvl1, results_shootarget_lvl2, results_shootarget_lvl3, results_shootarget_lvl4, results_shootarget_lvl5 = run_simulation(args)
# pass the results and args to the save function (correct argument order)
save_results_to_hdf5(store_data_filename, results_shootarget_lvl1, results_shootarget_lvl2, results_shootarget_lvl3, results_shootarget_lvl4, results_shootarget_lvl5, args)
#


(109, 109) (109, 100)


KeyboardInterrupt: 